# Reinforcement Learning Programming - CSCN8020
# Tessa Ayvazoglu
# 21/07/2024
# ============================================================================================

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objects as go
import matplotlib.pyplot as plt

import os

In [2]:
# Load dataframe
nvda_df1 = pd.read_csv(r'C:\Users\Admin\Documents\MLAI\CSCN8020RL\reinforcement_learning\FinalProject\NVDA_data.csv')

# Convert 'Date' column to datetime
nvda_df1['Date'] = pd.to_datetime(nvda_df1['Date'])

# Set 'Date' as index
nvda_df1 = nvda_df1.set_index('Date')

# Filter for July and August 2024
nvda_df = nvda_df1.loc['2024-07-01':'2024-08-31']

# Print or save the filtered dataframe to check
print(nvda_df.head())
nvda_df.to_csv('NVDA_data_filtered_July_August_2024.csv')

                  Open        High         Low       Close   Adj Close   
Date                                                                     
2024-07-01  123.470001  124.839996  118.830002  124.300003  124.300003  \
2024-07-02  121.129997  123.410004  121.029999  122.669998  122.669998   
2024-07-03  121.660004  128.279999  121.360001  128.279999  128.279999   
2024-07-05  127.379997  128.850006  125.680000  125.830002  125.830002   
2024-07-08  127.489998  130.770004  127.040001  128.199997  128.199997   

               Volume  
Date                   
2024-07-01  284885500  
2024-07-02  218374000  
2024-07-03  215749000  
2024-07-05  214176700  
2024-07-08  237677300  


In [3]:
# Show head of the dataframe
nvda_df.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2024-07-01,123.470001,124.839996,118.830002,124.300003,124.300003,284885500
2024-07-02,121.129997,123.410004,121.029999,122.669998,122.669998,218374000
2024-07-03,121.660004,128.279999,121.360001,128.279999,128.279999,215749000
2024-07-05,127.379997,128.850006,125.680000,125.830002,125.830002,214176700
2024-07-08,127.489998,130.770004,127.040001,128.199997,128.199997,237677300


## Evaluation of Q-Learning


### Softmax policy

#### Beta parameter


#### Alpha parameter


#### Gamma parameter
Let's now compare the various discount factors.

### Epsilon-greedy policy
#### Epsilon parameter
Now let's compare the different epsilons, it should be noted that when $\epsilon$ = 0 the policy is greedy

#### Alpha parameter
Now let's compare the different learning rates

### Gamma parameter
Now let's compare the different discount factors

## Evaluation of SARSA

### Softmax policy

#### Beta parameter


#### Alpha parameter
Now let's compare the different learning rates

#### Gamma parameter
Now let's compare the different discount factors

### Epsilon-greedy policy
#### Epsilon parameter
Now let's compare the different epsilons, it should be noted that when $\epsilon$ = 0 the policy is greedy

#### Alpha parameter
Now let's compare the different learning rates

### Gamma parameter
Now let's compare the different discount factors

In [4]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import random
import time
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# Define the Agent class
class Agent:
    def __init__(self, alpha, gamma, epsilon, beta, data, assets, policy, window_size):
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.beta = beta
        self.data = data
        self.assets = assets
        self.policy = policy
        self.window_size = window_size
        self.state_size = window_size
        self.action_size = 3  # sit, buy, sell
        self.memory = deque(maxlen=500)
        self.inventory = []

        self.model = self._model()

    def _model(self):
        model = Sequential()
        model.add(Dense(units=64, input_dim=self.state_size, activation="relu"))
        model.add(Dense(units=32, activation="relu"))
        model.add(Dense(units=8, activation="relu"))
        model.add(Dense(self.action_size, activation="linear"))
        model.compile(loss="mse", optimizer=Adam(learning_rate=0.001))
        return model

    def act(self, state):
        state = np.reshape(state, [1, self.state_size])  # Ensure correct shape
        if random.random() <= self.epsilon:
            return random.randrange(self.action_size)
        options = self.model.predict(state)
        return np.argmax(options[0])

    def expReplay(self, batch_size):
        if len(self.memory) < batch_size:
            return  # Not enough samples to perform experience replay

        mini_batch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in mini_batch:
            target = reward
            if not done:
                target = reward + self.gamma * np.amax(self.model.predict(np.reshape(next_state, [1, self.state_size]))[0])
            target_f = self.model.predict(np.reshape(state, [1, self.state_size]))
            target_f[0][action] = target
            self.model.fit(np.reshape(state, [1, self.state_size]), target_f, epochs=1, verbose=0)  # Suppress output

        if self.epsilon > 0.01:
            self.epsilon *= 0.995

    def simulate(self, max_episodes=20, max_time=120):
        rewards = []
        gains = []
        episodes = min(max_episodes, 10)
        start_time = time.time()
        
        for episode in range(episodes):
            if time.time() - start_time > max_time:
                print(f"Simulation terminated early at episode {episode} due to time limit.")
                break
            
            total_reward = 0
            state = self.data[:self.window_size]
            for i in range(self.window_size, len(self.data)):
                action = self.act(state)
                reward = 0  # Placeholder for reward calculation
                next_state = self.data[i-self.window_size:i]
                self.memory.append((state, action, reward, next_state, False))
                self.expReplay(batch_size=32)
                state = next_state
                total_reward += reward
                
                # Print key information
                print(f"Episode {episode}, Step {i}: Action {action}, Reward {reward}, Total Reward {total_reward}")

            rewards.append(total_reward)
            gains.append(total_reward)
            
            # Optional: Save results periodically
            if episode % 5 == 0:  # Less frequent saving
                pd.DataFrame(gains).to_csv('simulation_gains.csv', index=False)
        
        # Print final results
        print(f"Final Rewards: {rewards}")
        print(f"Final Gains: {gains}")
        return rewards, gains, None, None

def main():
    # Define smaller data window
    data_window = np.array(nvda_df[['Close']].rolling(window=5).mean().dropna())  # Reduced window size

    # Run simulations
    def run_simulations(parameters, policy_type):
        results = []
        for param in parameters:
            if policy_type == "softmax":
                beta = param
                agent = Agent(alpha=0.6, gamma=0.6, epsilon=0.2, beta=beta, data=data_window, assets=1000, policy="softmax", window_size=5)  # Reduced window size
            elif policy_type == "egreedy":
                epsilon = param
                agent = Agent(alpha=0.6, gamma=0.6, epsilon=epsilon, beta=4, data=data_window, assets=1000, policy="egreedy", window_size=5)  # Reduced window size
            else:
                raise ValueError("Policy type not recognized")

            # Simulate with reduced episodes and time
            rewards, gains, m, b = agent.simulate(max_episodes=20, max_time=120)  # Reduced episodes and time
            results.append((param, gains))
        
        return results

    # Define smaller parameter ranges
    beta_params = [0.5 * i for i in range(3)]  # Fewer beta parameters
    epsilon_params = [0.1 * i for i in range(5)]  # Fewer epsilon parameters

    # Run simulations
    softmax_results = run_simulations(beta_params, "softmax")
    egreedy_results = run_simulations(epsilon_params, "egreedy")

    # Plot results
    def plot_results(results, parameter_name, title):
        fig = go.Figure()
        
        for param, gains in results:
            fig.add_trace(go.Scatter(
                x=list(range(len(gains))),
                y=gains,
                mode='lines',
                name=f'{parameter_name} = {param}'
            ))
        
        fig.update_layout(
            title=title,
            xaxis_title='Episodes',
            yaxis_title='Money Gained',
            legend_title='Parameters'
        )
        
        fig.show()

    # Visualize results with a limited number of points
    plot_results(softmax_results, 'Beta', 'Total Gain per Session for Different Beta Values (Softmax Policy)')
    plot_results(egreedy_results, 'Epsilon', 'Total Gain per Session for Different Epsilon Values (E-Greedy Policy)')

    # Evaluate the agent in the context of RL
    def evaluate_agent(agent):
        # Example of evaluating the agent's actions
        state = data_window[:5]
        action = agent.act(state)
        print(f"Action taken: {action} (0: sit, 1: buy, 2: sell)")

        reward = 0  # Replace with actual reward logic
        print(f"Reward: {reward}")

        if reward > 0:
            print("Positive reward indicates a successful action in the given state.")
        else:
            print("Negative or zero reward indicates the action was not beneficial in the given state.")

    # Example usage of evaluate_agent with the last agent created (for demonstration purposes)
    if softmax_results:
        last_agent = Agent(alpha=0.6, gamma=0.6, epsilon=0.2, beta=beta_params[-1], data=data_window, assets=1000, policy="softmax", window_size=5)  # Last agent for example
        evaluate_agent(last_agent)
    if egreedy_results:
        last_agent = Agent(alpha=0.6, gamma=0.6, epsilon=epsilon_params[-1], beta=4, data=data_window, assets=1000, policy="egreedy", window_size=5)  # Last agent for example
        evaluate_agent(last_agent)

if __name__ == "__main__":
    main()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Episode 0, Step 5: Action 0, Reward 0, Total Reward 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step


c:\Users\Admin\Documents\MLAI\venv\venv\tensorflow_cpu\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Episode 0, Step 6: Action 0, Reward 0, Total Reward 0
Episode 0, Step 7: Action 1, Reward 0, Total Reward 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Episode 0, Step 8: Action 0, Reward 0, Total Reward 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Episode 0, Step 9: Action 0, Reward 0, Total Reward 0
Episode 0, Step 10: Action 2, Reward 0, Total Reward 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Episode 0, Step 11: Action 0, Reward 0, Total Reward 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Episode 0, Step 12: Action 0, Reward 0, Total Reward 0
Episode 0, Step 13: Action 2, Reward 0, Total Reward 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Episode 0, Step 14: Action 0, Reward 0, Total Reward 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Episode 0, Step 15: Action 0, Reward 0, Total Reward 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Episode 0, Step 16: Action 0, Reward 0, Total Reward 0
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Episode 0, Step 17: Action 0, Reward 0, Total Reward 0
Episode 0, Step 18: Action 1, Reward 0, 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


c:\Users\Admin\Documents\MLAI\venv\venv\tensorflow_cpu\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Action taken: 2 (0: sit, 1: buy, 2: sell)
Reward: 0
Negative or zero reward indicates the action was not beneficial in the given state.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
Action taken: 0 (0: sit, 1: buy, 2: sell)
Reward: 0
Negative or zero reward indicates the action was not beneficial in the given state.
